# **Real-Time ArUco Marker Detection and Pose Estimation using OpenCV**

## **Overview**
This script detects **ArUco markers** in a video stream, estimates their **position (X, Y, Z)** and **rotation (roll, pitch, yaw)**, and displays the information in real time.

## **Features**
- **ArUco Marker Detection** using `cv2.aruco.detectMarkers`
- **Pose Estimation** (3D position & rotation)
- **Rotation Matrix to Euler Angles Conversion**
- **Multi-threaded Processing** for efficient real-time performance
- **Live Display with Overlaid Text**
- **Exit on `Esc` key**

## **How It Works**
1. **Camera Setup**
   - The script initializes a webcam (`cv2.VideoCapture(0)`) to capture live video.
   - Defines a **camera matrix** and **distortion coefficients** (dummy values used here).

2. **Marker Detection & Pose Estimation**
   - Converts frames to **grayscale** for better detection.
   - Detects markers using **OpenCV's ArUco module**.
   - Draws detected markers and estimates:
     - **Position (X, Y, Z) in meters**
     - **Rotation (Roll, Pitch, Yaw) in degrees**

3. **Multi-threaded Processing**
   - Uses `threading` to separate:
     - **Capture & processing** (`capture_and_process`)
     - **Displaying frames** (`display_frame`)
   - The latest processed frame is stored in `latest_frame`.

4. **User Controls**
   - Press **Esc** to exit.

In [ ]:
import cv2
import numpy as np
import math
import threading

# Load the ArUco dictionary and parameters
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Open webcam
cap = cv2.VideoCapture(0)

# Define camera matrix and distortion coefficients (dummy values for demonstration)
# These values should ideally be obtained from camera calibration
camera_matrix = np.array([[1000, 0, 320],
                          [0, 1000, 240],
                          [0, 0, 1]], dtype=np.float32)
distortion_coefficients = np.zeros((5, 1), dtype=np.float32)

def rotation_matrix_to_euler_angles(R):
    """Convert a rotation matrix to Euler angles (roll, pitch, yaw)"""
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.degrees(np.array([x, y, z]))  # Convert to degrees

def capture_and_process():
    global latest_frame
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect ArUco markers in the image
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        # If markers are detected
        if len(corners) > 0:
            # Draw detected markers (without axes)
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Estimate pose of the markers
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, distortion_coefficients)

            # Loop over each detected marker
            for i in range(len(ids)):
                # Get translation vector (position)
                x, y, z = tvecs[i][0]
                position_text = f"Pos: ({x:.2f}, {y:.2f}, {z:.2f}) meters"
                cv2.putText(frame, position_text, (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                # Get rotation matrix from rotation vector
                R, _ = cv2.Rodrigues(rvecs[i])

                # Convert rotation matrix to Euler angles
                roll, pitch, yaw = rotation_matrix_to_euler_angles(R)
                rotation_text = f"Rot: ({roll:.2f}, {pitch:.2f}, {yaw:.2f}) deg"
                cv2.putText(frame, rotation_text, (10, 60 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Store the latest frame for display
        latest_frame = frame

def display_frame():
    while True:
        if latest_frame is not None:
            # Display the latest frame with overlaid text
            cv2.imshow('Frame', latest_frame)

        # Exit loop when 'Esc' key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Release the capture and close the window
    cap.release()
    cv2.destroyAllWindows()

# Shared variable for the latest frame
latest_frame = None

# Create and start threads
capture_thread = threading.Thread(target=capture_and_process)
display_thread = threading.Thread(target=display_frame)

capture_thread.start()
display_thread.start()

# Wait for threads to finish
capture_thread.join()
display_thread.join()
